<a href="https://colab.research.google.com/github/swarooptest/DOMAIN_EXTRACTOR_COMMON_CRAWL/blob/main/complete_script_common_crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMMON_CRAWL https://commoncrawl.org/get-started

**BASE URL FORMAT TO DOWNLOAD THE INDEXES/CDX FILES**                
                    https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/cc-index.paths.gz

**EXTRACT THE CDX URLS FROM THE BASE FILE**

In [ ]:

import gzip
import csv
import urllib.request
import os

def download_file(url, local_filename=None):
    """Download a file from a URL to a local file."""
    if local_filename is None:
        local_filename = url.split('/')[-1]

    print(f"Downloading {url} to {local_filename}...")

    if os.path.exists(local_filename):
        print(f"File {local_filename} already exists, skipping download.")
        return local_filename

    try:
        with urllib.request.urlopen(url) as response, open(local_filename, 'wb') as out_file:
            out_file.write(response.read())
        print(f"Download complete: {local_filename}")
        return local_filename
    except Exception as e:
        print(f"Error downloading file: {e}")
        return None

def extract_urls(paths_file):
    """Extract URLs from the cc-index.paths.gz file."""
    base_url = "https://data.commoncrawl.org"
    urls = []

    try:
        with gzip.open(paths_file, 'rt', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if line:
                    # Create the full URL by combining the base URL with the path
                    full_url = f"{base_url}/{line}"
                    urls.append(full_url)
    except Exception as e:
        print(f"Error processing file: {e}")

    return urls

def save_to_csv(urls, csv_filename="cc_index_urls.csv"):
    """Save URLs to a CSV file."""
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(csv_filename), exist_ok=True)

        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['URL'])  # Header
            for url in urls:
                writer.writerow([url])
        return True
    except Exception as e:
        print(f"Error saving to CSV: {e}")
        return False

def main():
    # URL of the cc-index.paths.gz file
    paths_url = "https://data.commoncrawl.org/crawl-data/CC-MAIN-2019-04/cc-index.paths.gz"

    # Extract crawl ID from URL (e.g., "CC-MAIN-2019-04")
    import re
    crawl_id_match = re.search(r'(CC-MAIN-\d{4}-\d{2})', paths_url)
    crawl_id = crawl_id_match.group(1) if crawl_id_match else "unknown-crawl"

    # Local filename to save the downloaded file
    local_file = "cc-index.paths.gz"

    # Download the file
    downloaded_file = download_file(paths_url, local_file)

    if downloaded_file:
        # Extract URLs from the file
        urls = extract_urls(downloaded_file)

        # Count and print the number of URLs
        url_count = len(urls)
        print(f"Found {url_count} URLs in the cc-index.paths.gz file")

        # Print the first few URLs as examples
        if url_count > 0:
            print("\nFirst 5 URLs (examples):")
            for i, url in enumerate(urls[:5]):
                print(f"{i+1}. {url}")

        # Create directory for crawl ID if it doesn't exist
        os.makedirs(crawl_id, exist_ok=True)

        # Save URLs to CSV with dynamic name based on crawl ID
        csv_filename = f"{crawl_id}/{crawl_id}_CC-INDEXES_CDX.csv"
        save_to_csv(urls, csv_filename)
        print(f"URLs saved to {csv_filename}")
    else:
        print("Failed to download or process the file")

if __name__ == "__main__":
    main()

Download complete: cc-index.paths.gz
Found 302 URLs in the cc-index.paths.gz file

First 5 URLs (examples):
1. https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00000.gz
2. https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00001.gz
3. https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00002.gz
4. https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00003.gz
5. https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00004.gz
URLs saved to CC-MAIN-2019-04/CC-MAIN-2019-04_CC-INDEXES_CDX.csv


**EXTRACT DOMAIN AND SUBDOMAIN FROM THE CDX URL "SINGLE URL"**

In [ ]:
import gzip
import json
import urllib.request
import os
from urllib.parse import urlparse
import time
import re
import csv
import socket
from concurrent.futures import ThreadPoolExecutor

def extract_domains_from_direct_url(url, output_file, display_limit=20):
    """
    Extract domain and subdomain information from a direct Common Crawl CDX file URL.

    Args:
        url: Direct URL to the CDX file
        output_file: Path to output file to save domains
        display_limit: Number of domains to display in console
    """
    domains = set()

    try:
        # Create a temporary file to store the downloaded gzipped content
        temp_file = "temp_cc_file.gz"

        # Download the file
        print(f"Downloading {url}")
        start_time = time.time()
        urllib.request.urlretrieve(url, temp_file)
        download_time = time.time() - start_time
        print(f"Download completed in {download_time:.2f} seconds")

        # Process the downloaded file
        print("Extracting domains...")
        start_time = time.time()
        domains = process_cdx_file(temp_file)
        processing_time = time.time() - start_time

        print(f"Extracted {len(domains)} unique domains")
        print(f"Processing completed in {processing_time:.2f} seconds")

        # Display some of the domains found
        if domains:
            print(f"\nSample of domains found (showing up to {display_limit}):")
            for i, domain in enumerate(sorted(domains)):
                if i >= display_limit:
                    print(f"...and {len(domains) - display_limit} more")
                    break
                print(f"  - {domain}")
            print()

        # Remove the temporary file
        os.remove(temp_file)

    except Exception as e:
        print(f"Error processing {url}: {e}")

    # Create the CSV file with domain and subdomain information
    create_domain_csv(domains, output_file)

    # Display some structured domains
    print("\n=== SAMPLE OF DOMAINS AND SUBDOMAINS ===")
    display_sample_from_csv(output_file, limit=display_limit)

    print(f"\nCompleted! Found {len(domains)} unique domains.")

    return domains

def process_cdx_file(file_path):
    """Process a single CDX file and return a set of domains"""
    domains = set()

    with gzip.open(file_path, 'rt', errors='replace') as f:
        for line_number, line in enumerate(f):
            try:
                # Split the line into SURT, timestamp, and JSON parts
                line = line.strip()

                # Find the JSON part - it starts after the second space
                parts = line.split(' ', 2)
                if len(parts) < 3:
                    continue

                surt = parts[0]
                json_str = parts[2]

                # Extract domain from JSON
                try:
                    record = json.loads(json_str)
                    url = record.get('url', '')

                    if url:
                        parsed_url = urlparse(url)
                        domain = parsed_url.netloc

                        if domain:
                            domains.add(domain)
                except json.JSONDecodeError:
                    # If JSON parsing fails, try to extract domain from SURT
                    match = re.match(r'^([^)]+)\)', surt)
                    if match:
                        surt_domain = match.group(1)
                        # Convert SURT domain to normal domain
                        domain_parts = surt_domain.split(',')
                        domain = '.'.join(reversed(domain_parts))
                        domains.add(domain)
            except Exception as e:
                if line_number % 100000 == 0:  # Only show occasional errors to reduce output
                    print(f"Error processing line {line_number}: {e}")
                continue

            # Show progress
            if line_number > 0 and line_number % 100000 == 0:
                print(f"Processed {line_number} lines, found {len(domains)} unique domains so far")

    return domains

def split_domain_parts(domain):
    """
    Split a domain into main domain and subdomain parts.
    Handles common multi-level TLDs.
    """
    parts = domain.split('.')

    # Handle common multi-part TLDs
    common_tlds = ['co.uk', 'com.au', 'co.nz', 'co.jp', 'co.za', 'ac.uk', 'gov.au', 'edu.au']

    if len(parts) >= 3:
        potential_tld = f"{parts[-2]}.{parts[-1]}"
        if potential_tld in common_tlds:
            # It's a multi-part TLD
            if len(parts) > 3:  # Has subdomain
                main_domain = f"{parts[-3]}.{parts[-2]}.{parts[-1]}"
                subdomain = '.'.join(parts[:-3])
                return main_domain, subdomain
            else:
                return f"{parts[-3]}.{parts[-2]}.{parts[-1]}", ""
        else:
            # Regular domain
            if len(parts) > 2:  # Has subdomain
                main_domain = f"{parts[-2]}.{parts[-1]}"
                subdomain = '.'.join(parts[:-2])
                return main_domain, subdomain
            else:
                return f"{parts[-2]}.{parts[-1]}", ""
    elif len(parts) == 2:
        # Just domain.tld
        return domain, ""
    else:
        # Invalid domain
        return domain, ""

def get_ip_address(domain):
    """Resolve domain to IP address"""
    try:
        ip = socket.gethostbyname(domain)
        return ip
    except socket.gaierror:
        return "Could not resolve"
    except Exception as e:
        return f"Error: {str(e)}"

def resolve_domain_batch(domains, max_workers=100):
    """Resolve a batch of domains to IP addresses using threading"""
    domain_to_ip = {}

    print(f"Resolving IP addresses for {len(domains)} domains (this may take some time)...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Create a mapping of futures to domains
        future_to_domain = {executor.submit(get_ip_address, domain): domain for domain in domains}

        # Process as they complete
        completed = 0
        for future in future_to_domain:
            domain = future_to_domain[future]
            try:
                ip = future.result()
                domain_to_ip[domain] = ip
            except Exception as e:
                domain_to_ip[domain] = f"Error: {str(e)}"

            completed += 1
            if completed % 100 == 0:
                print(f"Resolved {completed}/{len(domains)} domains")

    return domain_to_ip

def create_domain_csv(domains, output_file):
    """Create a CSV with domain, subdomain and IP address"""
    print(f"Creating CSV file: {output_file}")

    # Ensure directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)

    # Resolve IP addresses for all domains
    domain_to_ip = resolve_domain_batch(domains)

    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Full Domain', 'Main Domain', 'Subdomain', 'IP Address'])

        for full_domain in sorted(domains):
            main_domain, subdomain = split_domain_parts(full_domain)
            ip_address = domain_to_ip.get(full_domain, "Could not resolve")
            writer.writerow([full_domain, main_domain, subdomain, ip_address])

    print(f"CSV file created successfully with {len(domains)} entries")

def display_sample_from_csv(csv_file, limit=20):
    """Display a sample of domains from the CSV file"""
    try:
        with open(csv_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader)  # Skip header

            count = 0
            for row in reader:
                if count >= limit:
                    print(f"...and more entries (showing {limit} of total)")
                    break

                full_domain, main_domain, subdomain, ip_address = row

                print(f"Domain: {main_domain}")
                if subdomain:
                    print(f"  Subdomain: {subdomain}")
                print(f"  IP Address: {ip_address}")
                print()

                count += 1
    except Exception as e:
        print(f"Error displaying CSV contents: {e}")

# Main execution
if __name__ == "__main__":
    # Direct URL to the CDX file
    cdx_url = "https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2016-07/indexes/cdx-00045.gz"

    # Extract crawl ID from URL for dynamic output file naming
    import re
    crawl_id_match = re.search(r'(CC-MAIN-\d{4}-\d{2})', cdx_url)
    crawl_id = crawl_id_match.group(1) if crawl_id_match else "unknown-crawl"

    # Create directory for crawl ID if it doesn't exist
    os.makedirs(crawl_id, exist_ok=True)

    # Output file with dynamic naming based on crawl ID
    output_file = f"{crawl_id}/domains.csv"

    # Number of domains to display in console
    display_limit = 20

    # Extract domains from the specified CDX file
    extract_domains_from_direct_url(cdx_url, output_file, display_limit)

Download completed in 5.61 seconds
Extracting domains...
Processed 100000 lines, found 1554 unique domains so far
Processed 200000 lines, found 4830 unique domains so far
Processed 300000 lines, found 6495 unique domains so far
Processed 400000 lines, found 6552 unique domains so far
Processed 500000 lines, found 6552 unique domains so far
Processed 600000 lines, found 7081 unique domains so far
Processed 700000 lines, found 8308 unique domains so far
Processed 800000 lines, found 8909 unique domains so far
Processed 900000 lines, found 9622 unique domains so far
Processed 1000000 lines, found 9622 unique domains so far
Processed 1100000 lines, found 9622 unique domains so far
Processed 1200000 lines, found 9629 unique domains so far
Processed 1300000 lines, found 9629 unique domains so far
Processed 1400000 lines, found 9629 unique domains so far
Processed 1500000 lines, found 10369 unique domains so far
Processed 1600000 lines, found 12417 unique domains so far
Processed 1700000 line

KeyboardInterrupt: 

# Process URLs from a CSV file with column name "URL"
Handle errors gracefully

Auto-delete downloaded files after processing

Store extracted data from each file separately with proper naming

Allow the script to resume from where it left off if interrupted

Use a unique name based on the URL for resumption purposes

In [ ]:
import gzip
import json
import urllib.request
import os
from urllib.parse import urlparse
import time
import re
import csv
import socket
from concurrent.futures import ThreadPoolExecutor
import argparse
import hashlib
import logging
import sys
from google.colab import files  # For Colab file upload/download

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("domain_extractor.log"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

def create_progress_tracker(csv_file):
    """
    Create a progress tracker file to keep track of processed URLs

    Args:
        csv_file: Path to the input CSV file

    Returns:
        Path to the progress tracker file
    """
    base_name = os.path.basename(csv_file)
    name_without_ext = os.path.splitext(base_name)[0]
    tracker_file = f"{name_without_ext}_progress.txt"

    # Create the file if it doesn't exist
    if not os.path.exists(tracker_file):
        with open(tracker_file, 'w') as f:
            f.write("# URLs that have been processed\n")

    return tracker_file

def get_processed_urls(tracker_file):
    """
    Get the list of URLs that have already been processed

    Args:
        tracker_file: Path to the progress tracker file

    Returns:
        Set of URLs that have been processed
    """
    processed_urls = set()

    if os.path.exists(tracker_file):
        with open(tracker_file, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    processed_urls.add(line)

    return processed_urls

def mark_url_as_processed(tracker_file, url):
    """
    Mark a URL as processed in the tracker file

    Args:
        tracker_file: Path to the progress tracker file
        url: URL that has been processed
    """
    with open(tracker_file, 'a') as f:
        f.write(f"{url}\n")

def generate_url_filename(url):
    """
    Generate a unique filename based on the URL

    Args:
        url: URL to generate filename for

    Returns:
        A unique filename based on the URL
    """
    # Extract crawl ID and index number if possible
    crawl_id_match = re.search(r'(CC-MAIN-\d{4}-\d{2})', url)
    index_match = re.search(r'cdx-(\d+)\.gz', url)

    if crawl_id_match and index_match:
        crawl_id = crawl_id_match.group(1)
        index_num = index_match.group(1)
        return f"{crawl_id}_cdx_{index_num}"

    # Fallback to a hash of the URL
    url_hash = hashlib.md5(url.encode()).hexdigest()[:8]
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    file_part = path_parts[-1] if path_parts[-1] else 'index'

    return f"{parsed_url.netloc.replace('.', '_')}_{file_part}_{url_hash}"

def extract_domains_from_direct_url(url, output_file, display_limit=20):
    """
    Extract domain and subdomain information from a direct Common Crawl CDX file URL.

    Args:
        url: Direct URL to the CDX file
        output_file: Path to output file to save domains
        display_limit: Number of domains to display in console

    Returns:
        True if processing was successful, False otherwise
    """
    domains = set()
    temp_file = None
    success = False

    try:
        # Create a temporary file to store the downloaded gzipped content
        temp_file = f"temp_cc_file_{int(time.time())}.gz"

        # Download the file
        logger.info(f"Downloading {url}")
        start_time = time.time()

        try:
            urllib.request.urlretrieve(url, temp_file)
            download_time = time.time() - start_time
            logger.info(f"Download completed in {download_time:.2f} seconds")

        except Exception as e:
            logger.error(f"Failed to download {url}: {e}")
            return False

        # Process the downloaded file
        logger.info("Extracting domains...")
        start_time = time.time()

        try:
            domains = process_cdx_file(temp_file)
            processing_time = time.time() - start_time

            logger.info(f"Extracted {len(domains)} unique domains")
            logger.info(f"Processing completed in {processing_time:.2f} seconds")

            # Display some of the domains found
            if domains:
                logger.info(f"\nSample of domains found (showing up to {display_limit}):")
                for i, domain in enumerate(sorted(domains)):
                    if i >= display_limit:
                        logger.info(f"...and {len(domains) - display_limit} more")
                        break
                    logger.info(f"  - {domain}")

        except Exception as e:
            logger.error(f"Failed to process file {temp_file}: {e}")
            return False

    except Exception as e:
        logger.error(f"Error processing {url}: {e}")
        return False

    finally:
        # Remove the temporary file regardless of success or failure
        if temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
                logger.info(f"Deleted temporary file: {temp_file}")
            except Exception as e:
                logger.error(f"Failed to delete temporary file {temp_file}: {e}")

    # Create the CSV file with domain and subdomain information
    if domains:
        try:
            create_domain_csv(domains, output_file)

            # Display some structured domains
            logger.info("\n=== SAMPLE OF DOMAINS AND SUBDOMAINS ===")
            display_sample_from_csv(output_file, limit=display_limit)

            logger.info(f"\nCompleted! Found {len(domains)} unique domains.")
            success = True
        except Exception as e:
            logger.error(f"Failed to create CSV file {output_file}: {e}")
            return False
    else:
        logger.warning("No domains were extracted. Output file not created.")
        return False

    return success

def process_cdx_file(file_path):
    """Process a single CDX file and return a set of domains"""
    domains = set()

    with gzip.open(file_path, 'rt', errors='replace') as f:
        for line_number, line in enumerate(f):
            try:
                # Split the line into SURT, timestamp, and JSON parts
                line = line.strip()

                # Find the JSON part - it starts after the second space
                parts = line.split(' ', 2)
                if len(parts) < 3:
                    continue

                surt = parts[0]
                json_str = parts[2]

                # Extract domain from JSON
                try:
                    record = json.loads(json_str)
                    url = record.get('url', '')

                    if url:
                        parsed_url = urlparse(url)
                        domain = parsed_url.netloc

                        if domain:
                            domains.add(domain)
                except json.JSONDecodeError:
                    # If JSON parsing fails, try to extract domain from SURT
                    match = re.match(r'^([^)]+)\)', surt)
                    if match:
                        surt_domain = match.group(1)
                        # Convert SURT domain to normal domain
                        domain_parts = surt_domain.split(',')
                        domain = '.'.join(reversed(domain_parts))
                        domains.add(domain)
            except Exception as e:
                if line_number % 100000 == 0:  # Only show occasional errors to reduce output
                    logger.warning(f"Error processing line {line_number}: {e}")
                continue

            # Show progress
            if line_number > 0 and line_number % 100000 == 0:
                logger.info(f"Processed {line_number} lines, found {len(domains)} unique domains so far")

    return domains

def split_domain_parts(domain):
    """
    Split a domain into main domain and subdomain parts.
    Handles common multi-level TLDs.
    """
    parts = domain.split('.')

    # Handle common multi-part TLDs
    common_tlds = ['co.uk', 'com.au', 'co.nz', 'co.jp', 'co.za', 'ac.uk', 'gov.au', 'edu.au']

    if len(parts) >= 3:
        potential_tld = f"{parts[-2]}.{parts[-1]}"
        if potential_tld in common_tlds:
            # It's a multi-part TLD
            if len(parts) > 3:  # Has subdomain
                main_domain = f"{parts[-3]}.{parts[-2]}.{parts[-1]}"
                subdomain = '.'.join(parts[:-3])
                return main_domain, subdomain
            else:
                return f"{parts[-3]}.{parts[-2]}.{parts[-1]}", ""
        else:
            # Regular domain
            if len(parts) > 2:  # Has subdomain
                main_domain = f"{parts[-2]}.{parts[-1]}"
                subdomain = '.'.join(parts[:-2])
                return main_domain, subdomain
            else:
                return f"{parts[-2]}.{parts[-1]}", ""
    elif len(parts) == 2:
        # Just domain.tld
        return domain, ""
    else:
        # Invalid domain
        return domain, ""

def get_ip_address(domain):
    """Resolve domain to IP address with error handling"""
    try:
        ip = socket.gethostbyname(domain)
        return ip
    except socket.gaierror:
        return "Could not resolve"
    except Exception as e:
        return f"Error: {str(e)}"

def resolve_domain_batch(domains, max_workers=100):
    """Resolve a batch of domains to IP addresses using threading with error handling"""
    domain_to_ip = {}

    logger.info(f"Resolving IP addresses for {len(domains)} domains (this may take some time)...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Create a mapping of futures to domains
        future_to_domain = {executor.submit(get_ip_address, domain): domain for domain in domains}

        # Process as they complete
        completed = 0
        for future in future_to_domain:
            domain = future_to_domain[future]
            try:
                ip = future.result()
                domain_to_ip[domain] = ip
            except Exception as e:
                logger.warning(f"Error resolving {domain}: {e}")
                domain_to_ip[domain] = f"Error: {str(e)}"

            completed += 1
            if completed % 100 == 0:
                logger.info(f"Resolved {completed}/{len(domains)} domains")

    return domain_to_ip

def create_domain_csv(domains, output_file):
    """Create a CSV with domain, subdomain and IP address"""
    logger.info(f"Creating CSV file: {output_file}")

    # Ensure directory exists
    os.makedirs(os.path.dirname(output_file) if os.path.dirname(output_file) else '.', exist_ok=True)

    # Resolve IP addresses for all domains
    domain_to_ip = resolve_domain_batch(domains)

    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Full Domain', 'Main Domain', 'Subdomain', 'IP Address'])

        for full_domain in sorted(domains):
            main_domain, subdomain = split_domain_parts(full_domain)
            ip_address = domain_to_ip.get(full_domain, "Could not resolve")
            writer.writerow([full_domain, main_domain, subdomain, ip_address])

    logger.info(f"CSV file created successfully with {len(domains)} entries")

def display_sample_from_csv(csv_file, limit=20):
    """Display a sample of domains from the CSV file"""
    try:
        with open(csv_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader)  # Skip header

            count = 0
            for row in reader:
                if count >= limit:
                    logger.info(f"...and more entries (showing {limit} of total)")
                    break

                full_domain, main_domain, subdomain, ip_address = row

                logger.info(f"Domain: {main_domain}")
                if subdomain:
                    logger.info(f"  Subdomain: {subdomain}")
                logger.info(f"  IP Address: {ip_address}")
                logger.info("")

                count += 1
    except Exception as e:
        logger.error(f"Error displaying CSV contents: {e}")

def process_urls_from_csv(input_csv, output_dir="results", display_limit=20, start_url=None):
    """
    Process multiple URLs from a CSV file with resumption capability.

    Args:
        input_csv: Path to input CSV file containing URLs
        output_dir: Directory to store output files
        display_limit: Number of domains to display in console for each URL
        start_url: Optional URL to start processing from (for resumption)
    """
    logger.info(f"Processing URLs from {input_csv}")

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create progress tracker
    tracker_file = create_progress_tracker(input_csv)
    processed_urls = get_processed_urls(tracker_file)

    if processed_urls:
        logger.info(f"Found {len(processed_urls)} already processed URLs in tracker file")

    # Read URLs from CSV
    urls = []
    start_processing = False if start_url else True

    try:
        with open(input_csv, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)

            # Check if URL column exists
            if "URL" not in reader.fieldnames:
                logger.error("Column 'URL' not found in the CSV file")
                return

            # Get URLs from the URL column
            for row in reader:
                url = row["URL"].strip()
                if url and url.startswith("http"):
                    urls.append(url)

        logger.info(f"Found {len(urls)} URLs to process")

        # Process each URL
        for i, url in enumerate(urls):
            # Check if URL has already been processed
            if url in processed_urls:
                logger.info(f"[{i+1}/{len(urls)}] Skipping already processed URL: {url}")
                continue

            # If start_url is specified, skip until we find it
            if not start_processing:
                if url == start_url:
                    start_processing = True
                    logger.info(f"Starting from specified URL: {url}")
                else:
                    logger.info(f"[{i+1}/{len(urls)}] Skipping URL before start point: {url}")
                    continue

            logger.info(f"\n[{i+1}/{len(urls)}] Processing URL: {url}")

            # Generate output filename based on URL
            filename = generate_url_filename(url)
            output_file = f"{output_dir}/{filename}_domains.csv"

            # Process the URL
            success = extract_domains_from_direct_url(url, output_file, display_limit)

            if success:
                logger.info(f"Completed processing URL: {url}")
                logger.info(f"Results saved to: {output_file}")

                # Mark URL as processed
                mark_url_as_processed(tracker_file, url)
            else:
                logger.error(f"Failed to process URL: {url}")
                logger.info("You can resume from this URL later using the --start-url parameter")

    except Exception as e:
        logger.error(f"Error processing CSV file: {e}")

def upload_csv_file():
    """Allow user to upload a CSV file in Colab environment"""
    print("Please upload your CSV file with URLs...")
    uploaded = files.upload()

    if not uploaded:
        return None

    # Get the first uploaded file
    filename = list(uploaded.keys())[0]
    return filename

# Function to create a sample CSV if needed
def create_sample_csv(filename="sample_urls.csv"):
    """Create a sample CSV file with URLs"""
    logger.info(f"Creating sample CSV file: {filename}")

    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['URL'])
        writer.writerow(['https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00000.gz'])
        writer.writerow(['https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00001.gz'])
        writer.writerow(['https://data.commoncrawl.org/cc-index/collections/CC-MAIN-2019-04/indexes/cdx-00002.gz'])

    logger.info(f"Sample CSV file created with 3 URLs")
    return filename

def main():
    """Main function for Colab execution"""
    print("Common Crawl Domain Extractor")
    print("============================")

    # Options menu
    print("\nOptions:")
    print("1. Upload a CSV file with URLs")
    print("2. Create a sample CSV file")
    print("3. Specify a CSV file path")

    choice = input("\nEnter your choice (1-3): ")

    input_csv = None
    if choice == '1':
        input_csv = upload_csv_file()
        if not input_csv:
            print("No file uploaded. Exiting.")
            return
    elif choice == '2':
        input_csv = create_sample_csv()
    elif choice == '3':
        input_csv = input("Enter the path to your CSV file: ")
    else:
        print("Invalid choice. Exiting.")
        return

    # Get other parameters
    output_dir = input("Enter output directory (press Enter for 'results'): ")
    if not output_dir:
        output_dir = "results"

    display_limit_str = input("Enter display limit for samples (press Enter for default 20): ")
    display_limit = int(display_limit_str) if display_limit_str else 20

    start_url = input("Enter URL to start from (optional, press Enter to start from beginning): ")
    if not start_url:
        start_url = None

    # Process URLs
    print(f"\nProcessing URLs from {input_csv}")
    print(f"Results will be saved to {output_dir}")

    process_urls_from_csv(input_csv, output_dir, display_limit, start_url)

    print("\nProcessing complete!")
    print(f"Check the {output_dir} directory for results.")

# This will run when the script is executed directly in Colab
if __name__ == "__main__":
    # If running in Colab, use the interactive main function
    try:
        import google.colab
        # Running in Colab, use interactive interface
        main()
    except ImportError:
        # Not running in Colab, use command line arguments
        parser = argparse.ArgumentParser(description='Extract domains from Common Crawl CDX files')
        parser.add_argument('--input', '-i', type=str, required=True, help='Input CSV file containing URLs in column named "URL"')
        parser.add_argument('--output', '-o', type=str, help='Output directory', default='results')
        parser.add_argument('--limit', '-l', type=int, help='Display limit for domains', default=20)
        parser.add_argument('--start-url', '-s', type=str, help='URL to start processing from (for resumption)')

        args = parser.parse_args()

        logger.info("=== Domain Extractor ===")
        logger.info(f"Input file: {args.input}")
        logger.info(f"Output directory: {args.output}")
        logger.info(f"Display limit: {args.limit}")
        if args.start_url:
            logger.info(f"Starting from URL: {args.start_url}")

        # Process URLs from the CSV file
        process_urls_from_csv(args.input, args.output, args.limit, args.start_url)

# VALIDATE DOMAIN NAME AND IP ADDRESS FOR EACH DOWNLOADED CSV FILE

In [ ]:
import pandas as pd
import requests
import time
import json
from google.colab import files
import io
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import gc
import os
from datetime import datetime
import csv
import asyncio
import aiohttp
from collections import defaultdict
import hashlib

class FastOTXVerifier:
    def __init__(self, api_key, max_workers=20):
        self.api_key = api_key
        self.base_url = "https://otx.alienvault.com/api/v1"
        self.headers = {
            'X-OTX-API-KEY': api_key,
            'Content-Type': 'application/json'
        }
        self.max_workers = max_workers

        # Aggressive caching
        self.domain_cache = {}
        self.ip_cache = {}
        self.cache_hits = 0
        self.cache_misses = 0

        # Rate limiting - more aggressive
        self.request_semaphore = threading.Semaphore(15)  # 15 concurrent requests
        self.last_requests = []
        self.lock = threading.Lock()

        # Session pool
        self.session_pool = []
        for _ in range(max_workers):
            session = requests.Session()
            session.headers.update(self.headers)
            self.session_pool.append(session)
        self.session_index = 0

    def get_session(self):
        """Get session from pool"""
        with self.lock:
            session = self.session_pool[self.session_index]
            self.session_index = (self.session_index + 1) % len(self.session_pool)
            return session

    def rate_limit(self):
        """Smart rate limiting"""
        with self.lock:
            now = time.time()
            # Keep only requests from last second
            self.last_requests = [t for t in self.last_requests if now - t < 1.0]

            # If we've made too many requests, wait
            if len(self.last_requests) >= 10:  # 10 requests per second max
                sleep_time = 1.0 - (now - self.last_requests[0])
                if sleep_time > 0:
                    time.sleep(sleep_time)

            self.last_requests.append(now)

    def get_cache_key(self, item_type, item):
        """Generate cache key"""
        return f"{item_type}:{hashlib.md5(str(item).encode()).hexdigest()}"

    def check_domain_fast(self, domain):
        """Fast domain check with caching"""
        try:
            if not domain or str(domain).lower() in ['nan', 'null', '', 'none']:
                return {"status": "skipped", "pulse_count": 0, "threats": []}

            # Clean domain
            domain = str(domain).replace('http://', '').replace('https://', '').split('/')[0].strip().lower()
            if not domain:
                return {"status": "skipped", "pulse_count": 0, "threats": []}

            # Check cache first
            cache_key = self.get_cache_key('domain', domain)
            if cache_key in self.domain_cache:
                self.cache_hits += 1
                return self.domain_cache[cache_key]

            self.cache_misses += 1

            # Rate limit
            self.rate_limit()

            # Make request
            session = self.get_session()
            url = f"{self.base_url}/indicators/domain/{domain}/general"

            with self.request_semaphore:
                response = session.get(url, timeout=5)

            if response.status_code == 200:
                data = response.json()
                result = self.analyze_domain_response(data)
            elif response.status_code == 404:
                result = {"status": "safe", "pulse_count": 0, "threats": []}
            else:
                result = {"status": "error", "pulse_count": 0, "threats": []}

            # Cache result
            self.domain_cache[cache_key] = result
            return result

        except Exception as e:
            result = {"status": "error", "pulse_count": 0, "threats": []}
            # Cache errors too to avoid retrying
            cache_key = self.get_cache_key('domain', domain)
            self.domain_cache[cache_key] = result
            return result

    def check_ip_fast(self, ip):
        """Fast IP check with caching"""
        try:
            if not ip or str(ip).lower() in ['nan', 'null', '', 'none']:
                return {"status": "skipped", "pulse_count": 0, "threats": []}

            ip = str(ip).strip()
            if not self.is_valid_ip(ip):
                return {"status": "skipped", "pulse_count": 0, "threats": []}

            # Check cache first
            cache_key = self.get_cache_key('ip', ip)
            if cache_key in self.ip_cache:
                self.cache_hits += 1
                return self.ip_cache[cache_key]

            self.cache_misses += 1

            # Rate limit
            self.rate_limit()

            # Make request
            session = self.get_session()
            url = f"{self.base_url}/indicators/IPv4/{ip}/general"

            with self.request_semaphore:
                response = session.get(url, timeout=5)

            if response.status_code == 200:
                data = response.json()
                result = self.analyze_ip_response(data)
            elif response.status_code == 404:
                result = {"status": "safe", "pulse_count": 0, "threats": []}
            else:
                result = {"status": "error", "pulse_count": 0, "threats": []}

            # Cache result
            self.ip_cache[cache_key] = result
            return result

        except Exception as e:
            result = {"status": "error", "pulse_count": 0, "threats": []}
            # Cache errors too
            cache_key = self.get_cache_key('ip', ip)
            self.ip_cache[cache_key] = result
            return result

    def is_valid_ip(self, ip):
        """Fast IP validation"""
        try:
            parts = ip.split('.')
            return len(parts) == 4 and all(0 <= int(part) <= 255 for part in parts)
        except:
            return False

    def analyze_domain_response(self, data):
        """Fast domain analysis"""
        pulse_count = data.get('pulse_info', {}).get('count', 0)

        if pulse_count == 0:
            return {"status": "safe", "pulse_count": 0, "threats": []}

        # Quick threat detection
        pulses = data.get('pulse_info', {}).get('pulses', [])[:5]  # Only check first 5

        malicious_keywords = ['malware', 'phishing', 'botnet', 'ransomware', 'trojan']
        suspicious_keywords = ['suspicious', 'scan', 'probe']

        threats = []
        threat_level = "safe"

        for pulse in pulses:
            pulse_name = pulse.get('name', '').lower()
            tags = ' '.join(pulse.get('tags', [])).lower()

            # Quick keyword matching
            for keyword in malicious_keywords:
                if keyword in pulse_name or keyword in tags:
                    threat_level = "malicious"
                    threats.append(keyword)
                    break

            if threat_level != "malicious":
                for keyword in suspicious_keywords:
                    if keyword in pulse_name or keyword in tags:
                        threat_level = "suspicious"
                        threats.append(keyword)
                        break

        return {
            "status": threat_level,
            "pulse_count": pulse_count,
            "threats": list(set(threats))
        }

    def analyze_ip_response(self, data):
        """Fast IP analysis"""
        pulse_count = data.get('pulse_info', {}).get('count', 0)

        if pulse_count == 0:
            return {"status": "safe", "pulse_count": 0, "threats": []}

        # Quick threat detection
        pulses = data.get('pulse_info', {}).get('pulses', [])[:5]  # Only check first 5

        malicious_keywords = ['malware', 'botnet', 'c2', 'command', 'control']
        suspicious_keywords = ['suspicious', 'scan', 'probe']

        threats = []
        threat_level = "safe"

        for pulse in pulses:
            pulse_name = pulse.get('name', '').lower()
            tags = ' '.join(pulse.get('tags', [])).lower()

            # Quick keyword matching
            for keyword in malicious_keywords:
                if keyword in pulse_name or keyword in tags:
                    threat_level = "malicious"
                    threats.append(keyword)
                    break

            if threat_level != "malicious":
                for keyword in suspicious_keywords:
                    if keyword in pulse_name or keyword in tags:
                        threat_level = "suspicious"
                        threats.append(keyword)
                        break

        return {
            "status": threat_level,
            "pulse_count": pulse_count,
            "threats": list(set(threats))
        }

    def get_cache_stats(self):
        """Get cache statistics"""
        total = self.cache_hits + self.cache_misses
        hit_rate = (self.cache_hits / total * 100) if total > 0 else 0
        return {
            'hits': self.cache_hits,
            'misses': self.cache_misses,
            'hit_rate': hit_rate,
            'domain_cache_size': len(self.domain_cache),
            'ip_cache_size': len(self.ip_cache)
        }

def process_row_fast(verifier, row_data):
    """Process single row with optimizations"""
    domain = str(row_data.get('Full Domain', '')).strip() if pd.notna(row_data.get('Full Domain')) else ""
    ip_address = str(row_data.get('IP Address', '')).strip() if pd.notna(row_data.get('IP Address')) else ""

    # Quick skip for obviously empty data
    if not domain and not ip_address:
        return {
            'original_domain': domain,
            'ip_address': ip_address,
            'domain_status': 'skipped',
            'ip_status': 'skipped',
            'overall_risk': 'UNKNOWN'
        }

    # Process concurrently
    domain_result = verifier.check_domain_fast(domain) if domain else {"status": "skipped", "pulse_count": 0, "threats": []}
    ip_result = verifier.check_ip_fast(ip_address) if ip_address else {"status": "skipped", "pulse_count": 0, "threats": []}

    # Determine overall risk quickly
    if domain_result['status'] == 'malicious' or ip_result['status'] == 'malicious':
        overall_risk = 'HIGH'
    elif domain_result['status'] == 'suspicious' or ip_result['status'] == 'suspicious':
        overall_risk = 'MEDIUM'
    elif domain_result['status'] == 'safe' and ip_result['status'] == 'safe':
        overall_risk = 'LOW'
    else:
        overall_risk = 'UNKNOWN'

    return {
        'original_domain': domain,
        'main_domain': row_data.get('Main Domain', ''),
        'subdomain': row_data.get('Subdomain', ''),
        'ip_address': ip_address,
        'domain_status': domain_result['status'],
        'domain_threats': ', '.join(domain_result['threats']),
        'domain_pulse_count': domain_result['pulse_count'],
        'ip_status': ip_result['status'],
        'ip_threats': ', '.join(ip_result['threats']),
        'ip_pulse_count': ip_result['pulse_count'],
        'overall_risk': overall_risk
    }

def process_batch_parallel(verifier, batch_data, batch_num, total_batches):
    """Process batch with maximum parallelism"""
    print(f"🚀 Processing batch {batch_num}/{total_batches} ({len(batch_data)} rows)")

    results = []

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=verifier.max_workers) as executor:
        # Submit all rows in batch
        future_to_row = {
            executor.submit(process_row_fast, verifier, row): idx
            for idx, (_, row) in enumerate(batch_data.iterrows())
        }

        # Collect results as they complete
        for future in as_completed(future_to_row):
            try:
                result = future.result(timeout=10)
                results.append(result)
            except Exception as e:
                # Add error result
                results.append({
                    'original_domain': 'ERROR',
                    'ip_address': 'ERROR',
                    'domain_status': 'error',
                    'ip_status': 'error',
                    'overall_risk': 'ERROR'
                })

    return results

def main():
    print("⚡ ULTRA-FAST Domain & IP Security Verifier")
    print("🚀 Optimized for 300k-400k rows with aggressive caching")
    print("=" * 70)

    # Configuration for speed
    BATCH_SIZE = 2000  # Larger batches
    MAX_WORKERS = 20   # More workers
    CHECKPOINT_INTERVAL = 10000  # Less frequent checkpoints

    # Get API key
    api_key = input("Enter your AlienVault OTX API Key: ").strip()
    if not api_key:
        print("❌ API key is required!")
        return

    # Speed configuration
    print("\n⚡ Speed Configuration:")
    print(f"✅ Concurrent workers: {MAX_WORKERS}")
    print(f"✅ Batch size: {BATCH_SIZE}")
    print(f"✅ Aggressive caching: Enabled")
    print(f"✅ Rate limit: 10 req/sec per worker")

    # Initialize fast verifier
    verifier = FastOTXVerifier(api_key, max_workers=MAX_WORKERS)

    # Upload CSV
    print(f"\n📁 Upload your CSV file:")
    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded!")
        return

    filename = list(uploaded.keys())[0]

    try:
        start_time = time.time()

        # Fast CSV loading
        print("⚡ Fast loading CSV...")
        df = pd.read_csv(io.BytesIO(uploaded[filename]))

        total_rows = len(df)
        print(f"✅ Loaded {total_rows:,} rows in {time.time()-start_time:.1f}s")

        # Display column information
        print(f"\n📊 CSV Columns detected:")
        for i, col in enumerate(df.columns):
            print(f"  {i+1}. '{col}'")

        # Validate columns - FIXED: Check for correct column names
        required_columns = ['Full Domain', 'IP Address']
        missing_columns = [col for col in required_columns if col not in df.columns]

        if missing_columns:
            print(f"❌ Required columns missing: {missing_columns}")
            print(f"❌ Available columns: {list(df.columns)}")
            return

        print("✅ All required columns found!")

        # Memory optimization
        df = df.fillna('')  # Replace NaN with empty strings

        # Pre-analysis for duplicates
        print("🔍 Analyzing data for optimization...")
        unique_domains = df['Full Domain'].nunique()
        unique_ips = df['IP Address'].nunique()

        duplicate_savings = total_rows - unique_domains - unique_ips
        print(f"📊 Unique domains: {unique_domains:,}")
        print(f"📊 Unique IPs: {unique_ips:,}")
        print(f"💡 Cache will save ~{duplicate_savings:,} API calls")

        # Calculate batches
        total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
        print(f"\n🚀 Processing {total_batches} batches of {BATCH_SIZE} rows each")

        # Setup output
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"fast_verification_results_{timestamp}.csv"

        all_results = []
        processed_rows = 0

        # Process all batches
        for batch_num in range(1, total_batches + 1):
            batch_start_time = time.time()

            start_idx = (batch_num - 1) * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, total_rows)
            batch_data = df.iloc[start_idx:end_idx]

            # Process batch in parallel
            batch_results = process_batch_parallel(verifier, batch_data, batch_num, total_batches)
            all_results.extend(batch_results)
            processed_rows += len(batch_results)

            # Performance stats
            batch_time = time.time() - batch_start_time
            batch_rate = len(batch_results) / batch_time if batch_time > 0 else 0

            # Overall progress
            progress = (processed_rows / total_rows) * 100
            elapsed = time.time() - start_time
            overall_rate = processed_rows / elapsed if elapsed > 0 else 0
            eta = (total_rows - processed_rows) / overall_rate if overall_rate > 0 else 0

            # Cache stats
            cache_stats = verifier.get_cache_stats()

            print(f"✅ Batch {batch_num} complete: {batch_rate:.0f} rows/sec")
            print(f"📈 Overall: {progress:.1f}% ({processed_rows:,}/{total_rows:,}) - {overall_rate:.0f} rows/sec")
            print(f"⏱️ ETA: {eta/60:.1f} min | Cache hit rate: {cache_stats['hit_rate']:.1f}%")

            # Quick checkpoint for very large files
            if processed_rows % CHECKPOINT_INTERVAL == 0:
                checkpoint_file = f"checkpoint_{timestamp}.csv"
                pd.DataFrame(all_results).to_csv(checkpoint_file, index=False)
                print(f"💾 Checkpoint: {processed_rows:,} rows saved")

            # Memory management
            del batch_data, batch_results
            gc.collect()

        # Final results
        total_time = time.time() - start_time
        final_rate = processed_rows / total_time

        print(f"\n🎉 PROCESSING COMPLETE!")
        print(f"⚡ Total time: {total_time/60:.1f} minutes")
        print(f"🚀 Average rate: {final_rate:.0f} rows/minute")
        print(f"💾 Cache efficiency: {cache_stats['hit_rate']:.1f}% hit rate")

        # Quick summary
        results_df = pd.DataFrame(all_results)
        risk_counts = results_df['overall_risk'].value_counts()

        print(f"\n📊 SECURITY SUMMARY:")
        for risk, count in risk_counts.items():
            emoji = {"HIGH": "🔴", "MEDIUM": "🟡", "LOW": "🟢"}.get(risk, "⚪")
            print(f"{emoji} {risk}: {count:,} ({count/total_rows*100:.1f}%)")

        # Save and download
        print(f"\n💾 Saving results...")
        results_df.to_csv(output_file, index=False)

        print(f"📥 Downloading {output_file}...")
        files.download(output_file)

        print(f"\n✅ VERIFICATION COMPLETE!")
        print(f"🏆 Processed {total_rows:,} rows in {total_time/60:.1f} minutes")

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        # Emergency save
        if 'all_results' in locals() and all_results:
            emergency_file = f"emergency_save_{int(time.time())}.csv"
            pd.DataFrame(all_results).to_csv(emergency_file, index=False)
            files.download(emergency_file)
            print(f"💾 Emergency save: {len(all_results)} results")

if __name__ == "__main__":
    main()

⚡ ULTRA-FAST Domain & IP Security Verifier
🚀 Optimized for 300k-400k rows with aggressive caching

⚡ Speed Configuration:
✅ Concurrent workers: 20
✅ Batch size: 2000
✅ Aggressive caching: Enabled
✅ Rate limit: 10 req/sec per worker

📁 Upload your CSV file:


Saving CC-MAIN-2019-04_cdx_00000_domains.csv to CC-MAIN-2019-04_cdx_00000_domains.csv
⚡ Fast loading CSV...
✅ Loaded 340,333 rows in 0.7s

📊 CSV Columns detected:
  1. 'Full Domain'
  2. 'Main Domain'
  3. 'Subdomain'
  4. 'IP Address'
✅ All required columns found!
🔍 Analyzing data for optimization...
📊 Unique domains: 340,333
📊 Unique IPs: 106,325
💡 Cache will save ~-106,325 API calls

🚀 Processing 171 batches of 2000 rows each
🚀 Processing batch 1/171 (2000 rows)
✅ Batch 1 complete: 6 rows/sec
📈 Overall: 0.6% (2,000/340,333) - 6 rows/sec
⏱️ ETA: 881.9 min | Cache hit rate: 1.8%
🚀 Processing batch 2/171 (2000 rows)
✅ Batch 2 complete: 6 rows/sec
📈 Overall: 1.2% (4,000/340,333) - 6 rows/sec
⏱️ ETA: 887.0 min | Cache hit rate: 1.0%
🚀 Processing batch 3/171 (2000 rows)
✅ Batch 3 complete: 7 rows/sec
📈 Overall: 1.8% (6,000/340,333) - 7 rows/sec
⏱️ ETA: 846.8 min | Cache hit rate: 0.7%
🚀 Processing batch 4/171 (2000 rows)
✅ Batch 4 complete: 7 rows/sec
📈 Overall: 2.4% (8,000/340,333) - 7 r